## Global Commodity Trade ETL

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

### Extraction

In [2]:
# Extract the data from the trades table in the etl_python database
# Same values as the docker-compose file
HOST = "localhost"
PORT = 5433
DATABASE = "etl_python"
USER = "my_user"
PASSWORD = "my_password"

In [3]:
# Connect to the database
engine = create_engine(f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')

In [4]:
# By using a transaction, ensures that the connection is closed automatically 
# when the block is exited, even if an exception is raised.
with engine.begin() as conn:
    query = text("SELECT * FROM trades")
    # Batch loading the data (Number of rows)
    chunk_size = 1000
    # Create a pandas generator to read the data in batches
    trades_generator = pd.read_sql_query(query, conn, chunksize=chunk_size)
    # Concatenate batch DataFrames into a single one
    df_trades = pd.concat(trades_generator)

##### Trades 

In [5]:
df_trades.head(3)

,country_code,year,comm_code,flow,trade_usd,kg,quantity,quantity_name
0,SYC,1998,890200,Import,1431426.0,0.0,23000.0,Number of items
1,SYC,1998,890310,Import,31406.0,0.0,2545.0,Number of items
2,SYC,1998,890310,Export,950.0,0.0,300.0,Number of items


##### Countries

In [22]:
# Read the JSON file with the countries data
df_countries = pd.read_json('./sources/country_data.json')
df_countries.head(3)

,country,images_file,image_url,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,Flag_of_Afghanistan.svg,https://upload.wikimedia.org/wikipedia/commons...,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,None,142.0,34.0,NaN
1,Albania,Flag_of_Albania.svg,https://upload.wikimedia.org/wikipedia/commons...,AL,ALB,8.0,ISO 3166-2:AL,Europe,Southern Europe,None,150.0,39.0,NaN
2,Algeria,Flag_of_Algeria.svg,https://upload.wikimedia.org/wikipedia/commons...,DZ,DZA,12.0,ISO 3166-2:DZ,Africa,Northern Africa,None,2.0,15.0,NaN


##### Product codes 

In [14]:
# Read the CSV file with the products codes
df_product_codes = pd.read_csv('./sources/hs_codes.csv')
df_product_codes.head(3)

,Order,Level,Code,Parent,Code_comm,Parent.1,Description_complex,Description
0,1654555,1,0,NaN,NaN,NaN,This classification has been uploaded in RAMON...,This classification has been uploaded in RAMON...
1,1654556,1,10011000090,NaN,I,NaN,SECTION I - LIVE ANIMALS; ANIMAL PRODUCTS,LIVE ANIMALS; ANIMAL PRODUCTS
2,1654557,2,10021000090,1.001100e+10,1,I,CHAPTER 1 - LIVE ANIMALS,LIVE ANIMALS


In [9]:
# All product codes with Level == 2
df_parents = df_product_codes[df_product_codes['Level']==2].copy()
df_parents.head(3)

,Order,Level,Code,Parent,Code_comm,Parent.1,Description_complex,Description
2,1654557,2,10021000090,1.001100e+10,1,I,CHAPTER 1 - LIVE ANIMALS,LIVE ANIMALS
52,1654607,2,20021000090,1.001100e+10,2,I,CHAPTER 2 - MEAT AND EDIBLE MEAT OFFAL,MEAT AND EDIBLE MEAT OFFAL
140,1654695,2,30021000090,1.001100e+10,3,I,"CHAPTER 3 - FISH AND CRUSTACEANS, MOLLUSCS AND...","FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT..."


### Transformation

##### Clean product codes

In [15]:
# Only include the rows with a non-null value in the Code_comm column.
df_product_codes = df_product_codes[df_product_codes['Code_comm'].notnull()]
df_product_codes.head(3)

,Order,Level,Code,Parent,Code_comm,Parent.1,Description_complex,Description
1,1654556,1,10011000090,NaN,I,NaN,SECTION I - LIVE ANIMALS; ANIMAL PRODUCTS,LIVE ANIMALS; ANIMAL PRODUCTS
2,1654557,2,10021000090,1.001100e+10,1,I,CHAPTER 1 - LIVE ANIMALS,LIVE ANIMALS
3,1654558,3,10100000080,1.002100e+10,1.01,1,"Live horses, asses, mules and hinnies","Live horses, asses, mules and hinnies"


In [16]:
def get_clean_code(product_code):
    """
    The first 6 digits of the Code column in df_product_codes will later 
    allow us to match with trades in df_trades.

    The rows with a 1-digit Code_comm have an 11-digit Code column, 
    while the rest have a 12-digit Code column.
    
    Return: tuple(clean_code, parent_description)
    """
    
    product_code = str(product_code)

    # Define a dictionary to map code length to slice indices
    slice_indices = {11: (5, 1), 12: (6, 2)}

    # Get the appropriate slice indices based on the length of the code
    clean_product_code_slice_index, parent_code_slice_index = slice_indices[len(product_code)]
    
    # Slice the code to get the clean code and parent code
    clean_code = product_code[:clean_product_code_slice_index]
    parent_code = product_code[:parent_code_slice_index]

    try:
        # Only get the Description as a string
        parent_description = df_parents[df_parents['Code_comm']==parent_code]['Description'].values[0]
    except:
        parent_description = None
    return (clean_code, parent_description)

In [17]:
# Apply to each row the get_clean_code function
df_product_codes[['clean_code','parent_description']] = df_product_codes.apply(lambda x: get_clean_code(x['Code']), axis=1, result_type='expand')

In [18]:
# Only includes the rows with a non-null value in the clean_code column
# Then, it selects only three columns
df_product_codes = df_product_codes[df_product_codes['clean_code'].notnull()][['clean_code','Description','parent_description']]

In [19]:
# Create a unique identifier column
df_product_codes['id_code'] = df_product_codes.index

In [20]:
# Convert clean_code column to int
df_product_codes['clean_code'] = df_product_codes['clean_code'].astype('int64')
df_product_codes.head(3)

,clean_code,Description,parent_description,id_code
1,10011,LIVE ANIMALS; ANIMAL PRODUCTS,LIVE ANIMALS,1
2,10021,LIVE ANIMALS,LIVE ANIMALS,2
3,10100,"Live horses, asses, mules and hinnies",LIVE ANIMALS,3


##### Clean countries data

In [25]:
# The alpha-3 columns in df_countries will later allow us 
# to match with trades in df_trades.
df_countries = df_countries[['alpha-3', 'country', 'region', 'sub-region']]

In [26]:
# Only includes the rows with a non-null value in the alpha-3 column
df_countries = df_countries[df_countries['alpha-3'].notnull()]

In [27]:
# Create a unique identifier column 
# (+ 1, because the index of df_countries start at 0)
df_countries['id_country'] = df_countries.index + 1
df_countries.head(3)

,alpha-3,country,region,sub-region,id_country
0,AFG,Afghanistan,Asia,Southern Asia,1
1,ALB,Albania,Europe,Southern Europe,2
2,DZA,Algeria,Africa,Northern Africa,3


##### Merge dataframes

In [28]:
# Merge the df_trades Dataframe with the df_product_codes Dataframe
df_trades_clean = df_trades.merge(df_product_codes[['clean_code', 'id_code']],
                                  how='left',
                                  left_on='comm_code',
                                  right_on='clean_code')

In [29]:
# Merge the df_trades_clean Dataframe with the df_countries Dataframe
df_trades_clean = df_trades_clean.merge(df_countries[['alpha-3', 'id_country']],
                                  how='left',
                                  left_on='country_code',
                                  right_on='alpha-3')

In [31]:
def create_dimension(data, id_name):
    """
    Creates a new pandas DataFrame with an id column and a values column,
    based on the input data and id_name parameter.

    Args:
    - data: A pandas Series containing the values to be included in the 'values' column of the new DataFrame.
    - id_name: A string representing the name of the id column in the new DataFrame.

    Returns:
    - A pandas DataFrame with two columns: the specified id column and a 'values' column containing the input data.
    """
    
    keys_list = [i+1 for i in range(len(data))] 

    return pd.DataFrame({id_name:keys_list, 'values': data})

In [33]:
# Create the dimensions
df_quantity = create_dimension(df_trades_clean['quantity_name'].unique(), 'id_quantity')
df_flow = create_dimension(df_trades_clean['flow'].unique(), 'id_flow')
df_year = create_dimension(df_trades_clean['year'].unique(), 'id_year')

In [34]:
df_quantity.head(3)

,id_quantity,values
0,1,Number of items
1,2,Weight in kilograms
2,3,No Quantity


In [35]:
df_flow.head(3)

,id_flow,values
0,1,Import
1,2,Export
2,3,Re-Export


In [36]:
df_year.head(3)

,id_year,values
0,1,1998
1,2,1997
2,3,1996


In [37]:
# Merges the dataframes containing the dimension ids with df_trades_clean to 
# populate it with the corresponding dimensional values.
df_trades_clean = df_trades_clean.merge(df_quantity,
                                  how='left',
                                  left_on='quantity_name',
                                  right_on='values')
df_trades_clean = df_trades_clean.merge(df_flow,
                                  how='left',
                                  left_on='flow',
                                  right_on='values')
df_trades_clean = df_trades_clean.merge(df_year,
                                  how='left',
                                  left_on='year',
                                  right_on='values')

In [38]:
# Create a unique identifier column 
# (+ 1, because the index of df_trades_clean start at 0)
df_trades_clean['id_trades'] = df_trades_clean.index + 1

In [39]:
#Create df_trades_final, which will serve as our facts table. 
# This table will include unique identifiers, metrics such as trade_usd, kg, and quantity, 
# and corresponding ids to relate them with the dimensions.

df_trades_final = df_trades_clean[['id_trades', 'trade_usd', 'kg', 'quantity', 'id_code',
                                'id_country', 'id_quantity', 'id_flow', 'id_year']].copy()

In [53]:
df_trades_final.head()

,id_trades,trade_usd,kg,quantity,id_code,id_country,id_quantity,id_flow,id_year
0,1,1431426.0,0.0,23000.0,6928,155,1,1,1
1,2,31406.0,0.0,2545.0,6930,155,1,1,1
2,3,950.0,0.0,300.0,6930,155,1,2,1
3,4,950.0,0.0,300.0,6930,155,1,3,1
4,5,18251.0,0.0,450.0,6932,155,1,1,1


##### Rearrange the columns

In [40]:
df_countries = df_countries[['id_country','alpha-3', 'country', 'region', 'sub-region']]
df_product_codes = df_product_codes[['id_code','clean_code', 'Description', 'parent_description']]

##### Change columns names to match with the database

In [51]:
df_year.rename(columns = {'values':'year'}, inplace = True)
df_flow.rename(columns = {'values':'flow'}, inplace = True)
df_countries.rename(columns = {'alpha-3':'alpha_3', 'sub-region':'sub_region'}, inplace = True)
df_quantity.rename(columns = {'values':'quantity'}, inplace = True)
df_product_codes.rename(columns = {'clean_code':'code', 'Description':'description'}, inplace = True)

### Load

In [42]:
# Load to the target database trades
# Values for the trades database
HOST = "localhost"
PORT = 5433
DATABASE = "trades"
USER = "my_user"
PASSWORD = "my_password"

In [43]:
# Connect to the database
engine = create_engine(f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')

In [52]:
with engine.begin() as conn:
    # Load the DataFrames to the tables
    df_year.to_sql(name="years", con=conn, index=False, if_exists='append', chunksize=1000)
    df_flow.to_sql(name="flow", con=conn, index=False, if_exists='append', chunksize=1000)
    df_quantity.to_sql(name="quantity", con=conn, index=False, if_exists='append', chunksize=1000)
    df_countries.to_sql(name="countries", con=conn, index=False, if_exists='append', chunksize=1000)
    df_product_codes.to_sql(name="codes", con=conn, index=False, if_exists='append', chunksize=1000)
    df_trades_final.to_sql(name="trades", con=conn, index=False, if_exists='append', chunksize=1000)